In [6]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
from dynamic_object_detection.params import *
import yaml
import robotdatapy as rdp

time_tol = 1.0 # s
vel_estimate_dt = 0.25 # s
detected_threshold = 1.0 # m

In [7]:
os.environ['BAG_PATH'] = os.path.expanduser('/home/${USER}/Downloads/hamilton_cv_gt_2025-05-11.bag')
gt_bag = '~/Downloads/husky_gt/'

gt_topic = lambda obj: f'/{obj}/world'
cam = 'Husky'
objs = ['BD01', 'SCOUT2']

In [8]:
def project(obj_pos, cam_info):
    obj_pixel = cam_info['K'] @ obj_pos
    obj_pixel /= obj_pixel[2]
    obj_pixel = obj_pixel[:2]

    return obj_pixel

def within_cam_view(T_cam_obj, cam_info):
    obj_pos = T_cam_obj[:3, 3]

    if obj_pos[2] <= 0: return False

    obj_pixel = project(obj_pos, cam_info)

    return 0 <= obj_pixel[0] <= cam_info['W'] and \
           0 <= obj_pixel[1] <= cam_info['H']

def estimate_vel(pose_data, t, dt):
    idx1 = pose_data.idx(t - dt, force_single=True)
    idx2 = pose_data.idx(t + dt, force_single=True)

    if idx1 == idx2: return None

    t1 = pose_data.times[idx1]
    t2 = pose_data.times[idx2]
    pose1 = pose_data.pose(t1)
    pose2 = pose_data.pose(t2)

    return np.linalg.norm(pose2[:3, 3] - pose1[:3, 3]) / (t2 - t1)

def get_min_vel(params, z):
    return params.tracking_params.min_vel_threshold + z * params.tracking_params.vel_threshold_gain

def mahalanobis_distance(detected_obj_pos, detected_obj_cov, obj_pos):
    diff = detected_obj_pos - obj_pos
    return np.sqrt(diff.T @ np.linalg.inv(detected_obj_cov) @ diff)

In [9]:

def eval_run(run):

    # load data

    results_path = os.path.join('..', f'{run}.pkl')
    params_path = os.path.join('..', f'{run}.yaml')

    with open(results_path, 'rb') as f:
        results = pickle.load(f)

    params = Params.from_yaml(params_path)

    cam_pose_data = rdp.data.PoseData.from_bag(
        path=gt_bag,
        topic=gt_topic(cam),
        time_tol=time_tol,
        T_postmultiply=params.pose_data_params.T_odom_camera
    )

    obj_pose_data = [
        rdp.data.PoseData.from_bag(
            path=gt_bag,
            topic=gt_topic(obj),
            time_tol=time_tol,
        ) for obj in objs
    ]

    cam_info = results['camera_info']

    total_should_have_detected_objects = 0
    total_detected_objects = 0

    total_objects = 0
    total_correct_detections = 0

    total_detected_error = 0
    total_detected_error_count = 0

    for frame, time in enumerate(results['times']):

        try:
            cam_pose = cam_pose_data.pose(time)
            for i, obj in enumerate(objs):
                obj_pose = obj_pose_data[i].pose(time)
                T_cam_obj = np.linalg.inv(cam_pose) @ obj_pose

                if T_cam_obj[2, 3] > params.depth_data_params.max_depth: continue
                if not within_cam_view(T_cam_obj, cam_info): continue

                estimated_vel = estimate_vel(obj_pose_data[i], time, vel_estimate_dt)
        except:
            continue

        cam_pose = cam_pose_data.pose(time)

        frame_objs = results['objects'][frame]

        total_objects += len(frame_objs)

        correct_detection = [False] * len(frame_objs)

        for i, obj in enumerate(objs):

            obj_pose = obj_pose_data[i].pose(time)
            T_cam_obj = np.linalg.inv(cam_pose) @ obj_pose
            Z = T_cam_obj[2, 3]

            if Z > params.depth_data_params.max_depth: continue
            if not within_cam_view(T_cam_obj, cam_info): continue

            try:
                estimated_vel = estimate_vel(obj_pose_data[i], time, vel_estimate_dt)
            except:
                continue
            if estimated_vel is None: continue

            if estimated_vel >= get_min_vel(params, Z):

                false_negative = True

                for j, detected_obj in enumerate(frame_objs):
                    if np.linalg.norm(detected_obj['point'] - T_cam_obj[:3, 3]) < detected_threshold:
                        false_negative = False
                        correct_detection[j] = True

                        total_detected_error += np.linalg.norm(detected_obj['point'] - T_cam_obj[:3, 3])
                        total_detected_error_count += 1

                if false_negative:
                    total_should_have_detected_objects += 1
                else:
                    total_detected_objects += 1

        total_correct_detections += sum(correct_detection)

    print(f'Runtime: {results["runtime"]}')
    det_rate = total_detected_objects / (total_detected_objects + total_should_have_detected_objects)
    print(f'Out of {total_should_have_detected_objects + total_detected_objects} dynamic objects in frames, {total_detected_objects} were detected: {det_rate * 100:.2f}%. {(1- det_rate) * 100:.2f}% false negatives.')
    correct_rate = total_correct_detections / total_objects 
    print(f'Out of {total_objects} detected dynamic objects, {total_correct_detections} were correct detections: {correct_rate * 100:.2f}%. {(1 - correct_rate) * 100:.2f}% false positives.')
    print(f'Average error {total_detected_error / total_detected_error_count} m')
            

In [ ]:
# skip_runs = [f'out/skip_frames/hamilton_skip{n}' for n in (2, 3, 5)]
# raft_runs = [f'out/raft/hamilton_{model}' for model in ('kitti', 'things', 'sintel', 'chairs')]
# raft_12_runs = [f'out/raft/hamilton_{model}_12' for model in ('kitti', 'things', 'sintel', 'chairs')]
# ablation_runs = [f'out/ablations/no_{thing}' for thing in ('morph', 'stddev_prune', 'threshold_gain')]
test_3d_runs = ['out/3d/3d']

# runs = skip_runs + raft_runs + raft_12_runs + ablation_runs + test_3d_runs

# runs = [f'out/depths/{depth}' for depth in ('6', '7-5', '10', '15')]

runs = test_3d_runs

for run in runs:
    print(f'\n{run}\n')
    eval_run(run)


out/depths/6

Runtime: {'avg_batch_time': 2.1848832080441136, 'avg_frame_time': 0.0910368003351714, 'total_time': 270.9255177974701}
Out of 1283 dynamic objects in frames, 1088 were detected: 84.80%. 15.20% false negatives.
Out of 1384 detected dynamic objects, 1114 were correct detections: 80.49%. 19.51% false positives.
Average error 0.25054482210144874 m

out/depths/7-5

Runtime: {'avg_batch_time': 2.209628628146264, 'avg_frame_time': 0.09206785950609432, 'total_time': 273.9939498901367}
Out of 1306 dynamic objects in frames, 1087 were detected: 83.23%. 16.77% false negatives.
Out of 1434 detected dynamic objects, 1113 were correct detections: 77.62%. 22.38% false positives.
Average error 0.2772820460724708 m

out/depths/10

Runtime: {'avg_batch_time': 2.197720491117047, 'avg_frame_time': 0.09157168712987696, 'total_time': 272.5173408985138}
Out of 1306 dynamic objects in frames, 1049 were detected: 80.32%. 19.68% false negatives.
Out of 1317 detected dynamic objects, 1075 were cor